This is grammar is tailored to configuring API keys

program         -> service*
service         -> SERVICE_NAME '{' api_key_statement* '}'
api_key_statement -> 'api_key' '=' STRING ';'


an example of API Key configuration

google_maps {
    api_key = "YOUR_GOOGLE_MAPS_API_KEY";
}

weather_service {
    api_key = "YOUR_WEATHER_SERVICE_API_KEY";
}



In [ ]:
import re
import sys

In [ ]:
class Token:
    def __init__(self, type, value, position):
        self.type = type
        self.value = value
        self.position = position

class ASTNode:
    pass

In [ ]:
class ServiceNode(ASTNode):
    def __init__(self, service_name, api_keys, position):
        self.service_name = service_name
        self.api_keys = api_keys
        self.position = position

class ApiKeyNode(ASTNode):
    def __init__(self, key, value, position):
        self.key = key
        self.value = value
        self.position = position

In [ ]:
class ApiKeyNode(ASTNode):
    def __init__(self, key, value, position):
        self.key = key
        self.value = value
        self.position = position

In [ ]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = None
        self.index = 0

    def parse(self):
        services = []
        while self.index < len(self.tokens):
            service = self.parse_service()
            if service:
                services.append(service)
        return services

    def parse_service(self):
        position = self.current_token.position if self.current_token else None
        if self.match('SERVICE_NAME') and self.match('{'):
            service_name = self.previous().value
            api_key_statements = self.parse_api_key_statements()
            if self.match('}'):
                return ServiceNode(service_name, api_key_statements, position)
        return None

    def parse_api_key_statements(self):
        api_key_statements = []
        while not self.check('}'):
            if self.match('API_KEY'):
                position = self.current_token.position if self.current_token else None
                if self.match('='):
                    if self.match('STRING') and self.match(';'):
                        api_key_statements.append(ApiKeyNode("api_key", self.previous().value, position))
                    else:
                        self.error("Expected API key string followed by ';'", position)
                else:
                    self.error("Expected '=' after 'api_key'", position)
            else:
                self.error("Expected 'api_key' statement", position)
        return api_key_statements

    def match(self, expected_type):
        if self.check(expected_type):
            self.advance()
            return True
        return False

    def check(self, expected_type):
        return self.current_token.type == expected_type if self.current_token else False

    def advance(self):
        if self.index < len(self.tokens):
            self.current_token = self.tokens[self.index]
            self.index += 1

    def previous(self):
        return self.tokens[self.index - 1] if self.index > 0 else None

    def error(self, message, position=None):
        if position:
            raise Exception(f"Parse Error at position {position}: {message}")
        else:
            raise Exception(f"Parse Error: {message}")


In [ ]:
#Tokenizer function to convert code into tokens
def tokenize(code):
    token_specification = [
        ('SERVICE_NAME', r'[a-zA-Z_][a-zA-Z0-9_]*'),
        ('STRING', r'"(?:[^"\\]|\\.)*"'),
        ('EQUALS', r'='),
        ('SEMICOLON', r';'),
        ('OPEN_BRACE', r'{'),
        ('CLOSE_BRACE', r'}'),
        ('API_KEY', r'api_key'),
    ]

    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    tokens = []

    for match in re.finditer(tok_regex, code):
        for name, value in match.groupdict().items():
            if value:
                position = match.start(name) + 1
                tokens.append(Token(name, value, position))

    return tokens


In [ ]:
def is_valid_code(code):
    return all(char.isascii() for char in code)
code_input = input("Enter your configuration code:\n")

if not is_valid_code(code_input):
    print("Error: Invalid characters in the configuration code.")
    sys.exit(1)

tokens = tokenize(code_input)

print("\nTokens:")
for token in tokens:
    print(f"{token.type}: {token.value}")


Enter your configuration code:
something

Tokens:
SERVICE_NAME: something


In [ ]:
#parser
parser = Parser(tokens)

configurations = parser.parse()

print("\nParsed Configurations:")
for config in configurations:
    print(f"Service: {config.service_name}")
    for api_key_statement in config.api_keys:
        print(f"  API Key: {api_key_statement.key}, Value: {api_key_statement.value}")
ast = []
for config in configurations:
    service_name = config.service_name
    api_keys = []
    for api_key_statement in config.api_keys:
        key = api_key_statement.key
        value = api_key_statement.value
        api_keys.append(ApiKeyNode(key, value, api_key_statement.position))  # Append to api_keys list
    ast.append(ServiceNode(service_name, api_keys, config.position))

print("\nAST Structure:")
for node in ast:
    if isinstance(node, ServiceNode):
        print(f'Service: {node.service_name}')
        for api_key in node.api_keys:
            print(f'  Key: {api_key.key}, Value: {api_key.value}')

sys.exit(0)

